# Multiplier TEST code

fixed number to stochastic conversion & multiply & stochastic to fixed number conversion

## Import

In [1]:
import random, struct, math
import torch
import numpy as np
import torch.nn as nn 
import argparse
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

## Parser

In [2]:
parser = argparse.ArgumentParser(description='fixed_mac')
parser.add_argument('--full_bits', type=int, default=16, help='Number of Quantization Bits')
parser.add_argument('--frac_bits', type=int, default=8, help='Number of Quantization Bits')
parser.add_argument('--bBW', type=int, default=7, help='Number of bit width')
args = parser.parse_args(args=[])

## Definition

### int2bin : int number to binary number

In [3]:
def int2bin(iIn,iBW):
    iBW = iBW + 1
    if iIn >= 0:
        bOut = bin(iIn).replace('0b','').rjust(iBW,'0')
    else :
        bOut = bin(iIn & (pow(2,iBW)-1)).replace('0b','').rjust(iBW,'1')
    return bOut[1:]

### XOR : xor gate with string

In [4]:
def XOR(iA,iB):
    if iA != iB :
        iOut = '1'
    else : 
        iOut = '0'
    return iOut

### SNUM : sign number determination

In [5]:
def snum(a):
    if a >= 0 :
        return '0'
    else :
        return '1'

## binInv : binary Inversion

In [6]:
def binInv(bIn):
    bOut = bin(int(bIn,2)^(pow(2,len(bIn))-1)).replace('0b','').rjust(len(bIn),'0')
    return bOut

### fxp & flp2fix : floating number to fixed number conversion

In [7]:
class fxp:
    def __init__(self, bIn, iBWF):
        self.iFullBW = len(bIn)
        self.iIntgBW = self.iFullBW - iBWF
        self.bSign = bIn[0]
        self.bIntg = bIn[:self.iIntgBW]
        self.bFrac = bIn[self.iIntgBW:]
        self.fFull = 0
        try:
            for idx, bit in enumerate(bIn):
                if idx == 0:
                    self.fFull = self.fFull + int(bit,2) * -pow(2, self.iIntgBW - 1)
                else:
                    self.fFull = self.fFull + int(bit,2) * pow(2, self.iIntgBW - 1 - idx)
        except:
            print(bIn)
        self.dispFull = self.bIntg +"."+ self.bFrac 
        return

In [8]:
class flp2fix:
    def __init__(self, fIn, iBW, iBWF):
        self.fMin = - 2 ** (iBW - iBWF - 1)
        self.fMax = (2 ** (iBW-1) - 1) * (2 ** -iBWF)
        self.fResol = 2 ** -iBWF
        if fIn < self.fMin or fIn > self.fMax:
            print(f'({fIn}): Out of input range ({self.fMax}/{self.fMin}) during flp -> fix converting ')
        self.iBW = iBW
        self.iBWI = iBW - iBWF
        self.iBWF = iBWF

        self.iFLP2INT = abs(int(fIn * 2 ** iBWF))
        if fIn < 0:
            self.iFLP2INT = 2 ** (iBW-1) - self.iFLP2INT

        if fIn >= 0:
            self.bFull = bin(self.iFLP2INT)[2:].rjust(iBW, '0')
        else:
            self.bFull = '1'+bin(self.iFLP2INT)[2:].rjust(iBW-1, '0')
            if len(self.bFull) > iBW:
                self.bFull = '0' * iBW

        self.cssFxp = fxp(self.bFull, self.iBWF)
        self.bSign = self.cssFxp.bSign
        self.bIntg = self.cssFxp.bIntg
        self.bFrac = self.cssFxp.bFrac
        self.fFull = self.cssFxp.fFull
        return

In [9]:
def flp2fixTensor(fIn, iBW, iBWF):
    fMin = - 2 ** (iBW - iBWF - 1)
    fMax = (2 ** (iBW-1) - 1) * (2 ** -iBWF)
    fList = []
    for aTensor in fIn.view(-1):
        fList.append(flp2fix(aTensor, iBW, iBWF).fFull)
    return torch.tensor(fList).view(fIn.size())

### LFSR : make pseudorandom number bitstream

In [10]:
class LFSR7:
    def Random(self):
        self.b0 = eval(f'str(random.randint(0,1))')
        self.b1 = eval(f'str(random.randint(0,1))')
        self.b2 = eval(f'str(random.randint(0,1))')
        self.b3 = eval(f'str(random.randint(0,1))')
        self.b4 = eval(f'str(random.randint(0,1))')
        self.b5 = eval(f'str(random.randint(0,1))')
        self.b6 = eval(f'str(random.randint(0,1))')
        if int(self.b0) + int(self.b1) + int(self.b2) + int(self.b3) + int(self.b4) + int(self.b5) + int(self.b6) == 0 :
            self.b0 = eval(f'str(random.randint(0,1))')
            self.b1 = eval(f'str(random.randint(0,1))')
            self.b2 = eval(f'str(random.randint(0,1))')
            self.b3 = eval(f'str(random.randint(0,1))')
            self.b4 = eval(f'str(random.randint(0,1))')
            self.b5 = eval(f'str(random.randint(0,1))')
            self.b6 = eval(f'str(random.randint(0,1))')
        return self.b0 + self.b1 + self.b2 + self.b3 + self.b4 + self.b5 + self.b6
    
    def Normal(self,stream):
        self.b0 = XOR(int(stream[5]),int(stream[6]))
        self.b1 = stream[0]
        self.b2 = stream[1]
        self.b3 = stream[2]
        self.b4 = stream[3]
        self.b5 = stream[4]
        self.b6 = stream[5]
        
        return self.b0 + self.b1 + self.b2 + self.b3 + self.b4 + self.b5 + self.b6
    
    def Allzero(self):
        self.b0 = '0'
        self.b1 = '0'
        self.b2 = '0'
        self.b3 = '0'
        self.b4 = '0'
        self.b5 = '0'
        self.b6 = '0'
        
        return self.b0 + self.b1 + self.b2 + self.b3 + self.b4 + self.b5 + self.b6

### LFSRlist : make pseudorandom number bitstream with 2**bBW cycle

In [11]:
def LFSRlist7():
    lfsr = LFSR7()
    lfsrlist = []
    for k in range(2**(args.bBW)-1): #lfsr number generating
        if k == 0:
            lfsrlist.append(lfsr.Random())
        else :
            lfsrlist.append(lfsr.Normal(lfsrlist[k-1]))
        if (k == 2**(args.bBW)-2):
            lfsrlist.append(lfsr.Allzero())
    
    if (args.bBW) != args.frac_bits :
        if args.bBW < args.frac_bits :
            for i in range(len(lfsrlist)):
                lfsrlist[i] = lfsrlist[i] + (args.frac_bits-args.bBW)*'0'
        else :
            print("it can't work")
            return 0
    
    return lfsrlist

### Comp : Comparator in SNG

In [12]:
def Comp(a,lfsr,snum):
    for com in range(0,len(a)):
        oA = '0'
        if a[com]!=lfsr[com]:
            if(int(a[com]) > int(lfsr[com])):
                oA = '1'
            break
    return XOR(oA,snum)

### Perm : module for permutation SNG 

In [13]:
def perm(a):
    al = len(a)
    blist = []
    if args.frac_bits == args.bBW : 
        for i in range(al) :
            blist.append(a[al-i-1])
    elif args.frac_bits > args.bBW :
        for i in range(al-(args.frac_bits-args.bBW)) :
            blist.append((a[al-(args.frac_bits-args.bBW)-i-1]))
    b = "".join(blist)
    b = b + ('0'*(args.frac_bits-args.bBW))
    return b

### findMaxMin : find abs max value in act tensor

In [14]:
def findMaxMin(data):
    max = torch.max(data)
    min = torch.min(data)
    SF=torch.max(abs(max),abs(min)).item()
    return SF

### SNG : stochastic number generator module

In [15]:
def SNG(iIN,lfsr):

    sNUM = snum(iIN)
    
    bIN = flp2fix(iIN,args.full_bits,args.frac_bits).bFull
    bFRAC = bIN[-(args.frac_bits):]
    if sNUM == 1 :
        bFRAC = bin(int(binInv(bFRAC),2)+1).replace('0b','').rjust(args.bBW,'0')
    oAlist = []
    
    for k in range(2**(args.bBW)): #lfsr number generating
        lNUM = lfsr[k]
        a = Comp(bFRAC,lNUM,sNUM)
        oAlist.append(a) #comparator of input a
    
    oAlist.insert(0,sNUM)
    sA = "".join(oAlist)
    if bIN == args.full_bits*'0' :
        return '0'*(2**(args.bBW)+1)
    else :
        return sA

### SNG_P : permutation stochastic number generator module

In [16]:
def SNG_P(iIN,lfsr):
    sNUM = snum(iIN)
    
    bIN = flp2fix(iIN,args.full_bits,args.frac_bits).bFull
    bFRAC = bIN[-(args.frac_bits):]
    if sNUM == 1 :
        bFRAC = bin(int(binInv(bFRAC),2)+1).replace('0b','').rjust(args.bBW,'0')
    oAlist = []
    
    for k in range(2**(args.bBW)): #lfsr number generating    
        lNUM = perm(lfsr[k])
        a = Comp(bFRAC,lNUM,sNUM)
        oAlist.append(a) #comparator of input a
    
    oAlist.insert(0,sNUM)
    sA = "".join(oAlist)
    
    if bIN == args.full_bits*'0' :
        return '0'*(2**(args.bBW)+1)
    else :
        return sA

### SNGnumpy : SNG module with numpy

In [17]:
def SNGnumpy(fIn,lfsr):
    sList = []
    for aNumpy in fIn.view(-1):
        sList.append(SNG(float(aNumpy),lfsr))
                     
    return np.array(sList).reshape(fIn.size())

### SNGpnumpy : permutation SNG module with numpy

In [18]:
def SNGpnumpy(fIn,lfsr):
    sList = []
    for aNumpy in fIn.view(-1):
        sList.append(SNG_P(float(aNumpy),lfsr))
                     
    return  np.array(sList).reshape(fIn.size())

### CountOne : count 1 in stochastic number bit stream

In [19]:
def CountOne(nIn):
    nlist = []
    for num in nIn.reshape(-1):
        n = 0
        for a in num:
            if a == '1' :
                n += 1
        if num[0] == '1' :
            nlist.append(n-1)
        else :
            nlist.append(n)
    return torch.tensor(nlist).view(nIn.shape)

### defSign : sign determination in S2N 

In [20]:
def defSign(nIn):
    nlist = []
    for num in nIn.reshape(-1):
        if num[0] == '1' :
            nlist.append(-1)
        else :
            nlist.append(1)
    return torch.tensor(nlist).view(nIn.shape)

### Multiplier

In [21]:
def mul(a,b):
    al = len(a)
    bl = len(b)
    
    outlist = []
    
    if al != bl :
        print("length of string is different")
        return 0
    
    outlist.append(XOR(a[0],b[0]))
    
    for i in range(al-1) :
        outlist.append(str(int(a[i+1]) & int(b[i+1])))
    
    #print(outlist)
    out = "".join(outlist)
    
    return out    

In [22]:
def defSign1(nIn):
    if nIn[0] == '1' :
        return -1
    else :
        return 1

In [23]:
def CountOne1(nIn):
    n = 0
    for num in nIn:
        if num == '1' :
            n += 1
    if nIn[0] == '1' :
        return n-1
    else :
        return n

In [24]:
def S2None(sIn,SF):
    s = defSign1(sIn)
    o = (CountOne1(sIn)/(2**args.bBW))*SF*s
    return o

In [25]:
def mulNumpy(aIn,bIn,aSF,wSF):
    mList = []
    for i in range(aIn.shape[0]):
        for j in range(bIn.T.shape[1]):
            sum = 0
            for k in range(aIn.shape[1]):
                sum += S2None(mul((aIn[i][k].astype(str)),(bIn.T)[k][j].astype(str)),aSF*wSF)
            mList.append(sum)
    return torch.tensor(mList).view(aIn.shape[0],bIn.T.shape[1])

In [26]:
def FindError(a,b):
    elist = []
    for anum,bnum in zip(a.view(-1),b.view(-1)):
        e = (abs(bnum-anum)*100).item()
        elist.append(e)
    return torch.tensor(elist).reshape(a.size())

# FC layer

In [47]:
def fc(aIn,bIn):
    out = torch.matmul(aIn,bIn.t())
    return out

# TEST : LFSR 7 bit

In [28]:
act = torch.rand((100,150))-torch.randint(5,(100,150))+torch.randint(5,(100,150))
wei = torch.rand((100,150))-torch.randint(5,(100,150))+torch.randint(5,(100,150))

In [29]:
print(act)

tensor([[-4.2884e-01,  2.9331e+00, -1.7039e+00,  ..., -4.8921e-01,
         -9.9194e-01, -3.4655e-01],
        [-3.7130e-01,  6.5141e-01,  1.0435e+00,  ...,  1.6701e+00,
         -1.9693e+00,  2.6685e+00],
        [ 2.1331e+00,  3.2336e+00, -6.6079e-01,  ...,  1.7495e+00,
         -1.3973e+00,  9.3834e-01],
        ...,
        [-8.4405e-01,  3.0651e+00,  3.3237e+00,  ..., -1.4842e+00,
         -8.6900e-01,  4.0025e+00],
        [ 3.3059e+00,  3.6785e+00, -2.0764e+00,  ...,  1.8722e+00,
          2.2986e+00, -2.1710e+00],
        [ 2.8012e-01,  3.2067e+00,  2.3790e-03,  ...,  8.7790e-01,
          3.2359e+00,  2.4288e+00]])


In [30]:
print(wei)

tensor([[ 4.0946, -0.3852, -1.7794,  ..., -1.5075,  0.0754,  1.7726],
        [ 0.9166,  0.6981,  0.6144,  ..., -3.7006,  0.9094,  0.9224],
        [ 3.7833,  2.5138, -2.7625,  ...,  3.2481,  0.3966,  1.2035],
        ...,
        [ 0.1968,  0.6411, -2.1927,  ...,  0.0615, -1.0230, -3.2140],
        [ 4.3901,  3.7693, -0.8530,  ...,  1.1861, -2.6019, -2.4254],
        [ 3.3628,  2.6675, -0.3804,  ...,  3.0260, -1.8897, -1.3070]])


In [31]:
o = fc(act,wei)

In [32]:
print(o)
print(o.size())

tensor([[ 36.1159,   7.9622,  56.5647,  ..., -17.7908,  70.9501,  96.0673],
        [124.9998,  39.3227, -22.9943,  ...,  95.9322,  82.1913, -17.9430],
        [ -2.2882,   6.4043,  51.9475,  ..., 113.5482,  55.8708,  32.8936],
        ...,
        [ 94.3628, -11.5020,  22.7657,  ...,  43.1580,  25.3722,  96.5639],
        [-31.1977,  97.5132,  96.2715,  ..., 149.8351,  98.0473,  81.9889],
        [ 82.0090,  69.8969,  77.4719,  ..., -96.2086, 121.5263,  24.6192]])
torch.Size([100, 100])


In [33]:
act_f = flp2fixTensor(act,args.full_bits,args.frac_bits)
wei_f = flp2fixTensor(wei,args.full_bits,args.frac_bits)

### number to stochastic conversion

In [34]:
aSF = findMaxMin(act_f)
wSF = findMaxMin(wei_f)

In [35]:
def N2S(act,wei,aSF,wSF):
    
    lfsrlist = LFSRlist7()
    
    a = SNGnumpy(act/aSF,lfsrlist)
    w = SNGpnumpy(wei/wSF,lfsrlist)
    
    return a,w

In [36]:
def N2Sone(data,SF):
    lfsrlist = LFSRlist7()
    
    o = SNGnumpy(data/SF,lfsrlist)
    
    return o

### stochastic to number conversion

In [37]:
a,w = N2S(act,wei,aSF,wSF)

In [38]:
out = mulNumpy(a,w,aSF,wSF)

In [39]:
print(out.size())

torch.Size([100, 100])


In [40]:
#error = FindError(cc,act_f*wei_f)

In [41]:
#print(f'error mean value is : {torch.mean(error)}')
#print(f'------------------------------------------------------------------------------------------')
#for i in range(100):
#    for j in range(150):
#        print(f'act value is            : {act_f[i][j]:^25}( --> {ca[i][j]})')
#        print(f'wei value is            : {wei_f[i][j]:^25}( --> {cw[i][j]})')
#        print(f'stochastic mul value is : {cc[i][j]:^25}( --> {ca[i][j] * cw[i][j]})')
#        print(f'fixed mul value is      : {act_f[i][j]*wei_f[i][j]:^25}')
#        print(f'error is                : {error[i][j]:^25}')
#        print(f'------------------------------------------------------------------------------------------')

In [42]:
print(f'------------------------------------------------------------------------------------------')
for i in range(100):
    for j in range(100):
        print(f'stochastic FC value is            : {out[i][j]:^25}')
        print(f'original FC value is              : {o[i][j]:^25}')
        print(f'------------------------------------------------------------------------------------------')

------------------------------------------------------------------------------------------
stochastic FC value is            :    62.987403869628906    
original FC value is              :     36.11586380004883    
------------------------------------------------------------------------------------------
stochastic FC value is            :    34.906333923339844    
original FC value is              :     7.962160110473633    
------------------------------------------------------------------------------------------
stochastic FC value is            :     83.85320281982422    
original FC value is              :     56.56473922729492    
------------------------------------------------------------------------------------------
stochastic FC value is            :     65.91251373291016    
original FC value is              :     41.09092330932617    
------------------------------------------------------------------------------------------
stochastic FC value is            :     7.0202679

## ERROR

In [45]:
def MSE(a,b):
    s = 0
    n = 0

    for i in range(a.size()[0]):
        for j in range(b.size()[0]):
            square = (((a[i][j])-b[i][j])**2).item()
            s += square
            n += 1
    error = s/n
    return error

## ERROR : 7bit LFSR

In [46]:
MSE(out,o)

704.1379449159652